<a href="https://colab.research.google.com/github/oliverzannino/oliverza/blob/main/37005_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [84]:
#Q1a)

q1 = pd.read_csv('Q1.csv')
q1 = q1.drop('Unnamed: 0', axis=1)
q1 = q1.reset_index().rename(columns={'index': 'Asset'})
q1

,Asset,State 1,State 2,State 3,State 4,State 5,Time 0 price
0,0,92.08,5.31,72.58,32.53,44.47,41.525837
1,1,84.53,66.84,1.78,26.65,36.58,35.615789
2,2,25.47,15.00,92.71,38.92,94.38,53.512440
3,3,19.82,89.66,86.64,8.30,31.83,37.597608
4,4,40.46,73.06,77.39,3.42,10.37,29.093780
5,5,39.51,65.00,30.01,39.96,2.15,31.316746
6,6,2.99,55.57,1.78,27.51,56.89,31.342105
7,7,13.10,76.24,90.88,35.56,62.42,52.085167


In [65]:
#Set up the key equation that P = stateprices (psi) * payoffs (B)

# Because we have more equations (8) than unknowns/states (5), no exact state prices solves this perfectly. So we find the
# ψ that minimizes the total squared difference between predicted prices Bψ and actual prices P (least squares method).

P = q1[['Time 0 price']].values
B = q1[['State 1', 'State 2', 'State 3', 'State 4', 'State 5']].values

# Solve for state prices psi using least squares
psi, residuals, rank, s = np.linalg.lstsq(B, P, rcond=None)

# Compute the price of the risk-free asset
P_rf = np.sum(psi)

# Calculate the risk-free rate
r = 1 / P_rf - 1

# Output the result
print(f"State prices (psi): {psi}")
print(f"Price of risk-free asset (P_rf): {P_rf:6f}")
print(f"Risk-free rate (r): {r*100:.2f}%")
print(f"Residuals: {residuals}")


State prices (psi): [[0.09569378]
 [0.14354067]
 [0.14354067]
 [0.33492823]
 [0.23923445]]
Price of risk-free asset (P_rf): 0.956938
Risk-free rate (r): 4.50%
Residuals: [2.83989926e-28]


In [66]:
# The above is equivalent to using the matrix method using normal equations. psi = (BT * B)^-1 * BT * P
B_T = B.T
psi_matrix = np.linalg.inv(B_T @ B) @ B_T @ P

print("State prices via matrix inversion:", psi_matrix.flatten())

sum_psi_matrix = np.sum(psi_matrix)
print(f"State prices (psi): {sum_psi_matrix:.6f}")

#therefore

rf = 1/(sum_psi_matrix) - 1
print(f"RISK FREE RATE (Rf): {rf*100:.2f}%")

State prices via matrix inversion: [0.09569378 0.14354067 0.14354067 0.33492823 0.23923445]
State prices (psi): 0.956938
RISK FREE RATE (Rf): 4.50%


In [67]:
# additional check
Prices = B @ psi #reconstructed prices
originalprices = P # original prices

difference = originalprices - Prices

print("Difference:", difference)
print("Are differences close to zero?", np.allclose(difference, 0))

# Final answer: Risk free rate is 4.5%

Difference: [[2.13162821e-14]
 [1.42108547e-14]
 [7.10542736e-15]
 [2.13162821e-14]
 [1.77635684e-14]
 [1.77635684e-14]
 [7.10542736e-15]
 [2.13162821e-14]]
Are differences close to zero? True


#Q1b
If there is a 2% rate that we can borrow/lend at, and investing in the risk free asset yields 4.5%, the riskless profit strategy is to:  

T = 0:
*   Borrow 0.9569 at 2%
*   Invest in risk free asset (said another way, buy risk free asset for 0.9569)

T = 1:
*   Receive 1 from the risk free asset (4.5% yield)
*   Repay 0.9569*1.02 = 0.9760

Riskless profit is: $1 - $0.9760 = $0.0240

This way, cash flow at T = 0 is zero (+0.9569 and - 0.9569 offset) and cash flow at T = 1 is positive ($1 - $0.9760 = $0.0240).

This arbitrage strategy costs nothing at T = 0 and guarantees positive payoff in the future at all states i.e. generates riskless profit.

You can scale this strategy by N units to make sure it pays off $0.0240 * N in all possible states.

Rationale: Because the borrowing rate of 2% is lower than the implied risk-free rate of 4.5%, you can borrow money at 2%, invest in the risk-free portfolio that yields 4.5%, and at maturity receive 1 unit while only repaying less than 1 unit. This yields a riskless profit equal to the difference between the risk free payoff (1) and the repayment (0.9760), equal to $0.0240.





In [10]:
#q2

q2 = pd.read_csv('Q2.csv')
q2 = q2.drop('Unnamed: 0', axis=1)
q2 = q2.reset_index().rename(columns={'index': 'Asset'})
q2

,Asset,State 1,State 2,State 3,State 4,State 5,Time 0 price
0,0,50.70,82.89,6.40,6.32,44.72,31.230882
1,1,48.80,88.53,19.92,63.89,32.36,50.587255
2,2,16.40,37.92,57.80,5.49,76.63,36.350000
3,3,68.92,97.74,2.36,88.30,81.26,71.693137
4,4,0.92,51.06,43.19,52.36,29.06,39.039706
5,5,24.52,7.05,32.15,1.47,21.47,13.935294
6,6,4.82,70.89,82.88,84.00,8.34,55.095400
7,7,12.60,9.98,86.37,98.64,51.43,61.856863


In [15]:
P1 = q2[['Time 0 price']].values
B1 = q2[['State 1', 'State 2', 'State 3', 'State 4', 'State 5']].values

#least squares
psi1, residuals1, rank1, s1 = np.linalg.lstsq(B1, P1, rcond=None)

# Print results
print("State price vector psi1:", psi1)

##matrix (equivalent result)

B1_T = B1.T
psi_matrix1 = np.linalg.inv(B1_T @ B1) @ B1_T @ P1

print("State prices via matrix inversion:", psi_matrix1.flatten())

#Check whether the state prices * psi1 = original prices. If not, and |residual| >0, we have a mispricing
resid = P1 - B1 @ psi1

print(resid)

# If the residual ($||B\psi - P||^2$) from the least squares calculation is greater than zero, this indicates that there is no exact solution for state prices that perfectly prices all assets. This inconsistency in pricing hints at the possibility of arbitrage opportunities within the market.
print("Residual ||b1 @ psi1 - p1||^2:", residuals1)



State price vector psi1: [[0.09594138]
 [0.15309145]
 [0.1532252 ]
 [0.34287793]
 [0.23688839]]
State prices via matrix inversion: [0.09594138 0.15309145 0.1532252  0.34287793 0.23688839]
[[-0.06437461]
 [-0.27229578]
 [ 0.07975927]
 [ 0.23041569]
 [-0.32027136]
 [-0.01269786]
 [ 0.30360942]
 [-0.11856119]]
Residual ||b1 @ psi1 - p1||^2: [0.34671243]


In [91]:
# since these state prices do not perfectly match up with the prices (seen in the |resid| > 0), we have found an arbitrage solution.

# this means no state prices can perfectly price all assets.
# implying the market is not arbitrage-free, and we've found a riskless profit opportunity using the residual of the least squares solution.

# Remember, arbitrage can be either of the form of pay nothing now, receive positive payoff later (type 1), or
# receive money now, non-negative payoff in future (type 2)

In [73]:
# recall residual (mispricing)
resid = P1 - B1 @ psi1

# Define the abitrage direction such that you receive money up front (short the asset with positive residual, buy the asset with negative residual). T
xi = -resid

# Then we set up the conditions for the arbitrage portfolio. 1) xi.T @ B1 = 0, and 2) xi.T @ P1 < 0

# Calculate the vector of payoffs across states.
# Check if the payoff in all states is zero (recall the residual r is orthogonal to the column space of B1 ie B1T*resid = 0, property of least squares).
payoffallstate = xi.T @ B1 # xi = -resid.T * B1... but from the above property, this is just equal to 0.T = 0
payoffallstate

print("Payoff in all states:", payoffallstate)
print("Is payoff approximately zero in all future states?", np.allclose(payoffallstate, 0, atol=1e-8)) # need to ensure xi.T *B1 = 0. Satisfies

# Check if the initial cost is negative. Initial cost is equal to xi.T @ P1... same as -resid.T @ P1
# Recall P1 = B1 @ psi1 + resid. So Resid.T @ P1 = Resid.T (B1 @ psi1+resid)... given the orthogonal condition, just left with Resid.T @ P1 = Resid.T * Resid = ||Resid||^2
# so initial cost = -||Resid||^2. Need to ensure xi.T @ P1 <0 (we receive money)
initial_cost = xi.T @ P1
print("Initial cost:", initial_cost) # this is just the sum of squared residual from the least squares!
print("Is initial cost negative?", initial_cost < 0)

Payoff in all states: [[-4.15312229e-12 -3.21875859e-12  2.14228635e-12  1.16173737e-12
  -3.54560825e-12]]
Is payoff approximately zero in all future states? True
Initial cost: [[-0.34671243]]
Is initial cost negative? [[ True]]


In [76]:
#visualising in table format. # df for $0.3467 today, 0 payoff in future

weights = [-0.06437461*-1, -0.27229578*-1, 0.07975927*-1, 0.23041569*-1, -0.32027136*-1, -0.01269786*-1, 0.30360942*-1, -0.11856119*-1] # weights according to xi (-resid vector) as laid out above
q2df = pd.DataFrame({
    'Asset': range(8),   # Asset 0 through 7
    'Weight': weights

})

q2df['Current Price'] = P1
q2df['Cost Contribution'] = q2df['Weight']*q2df['Current Price']

print(f"Current cost of portfolio: {q2df['Cost Contribution'].sum():.4f}") # same as sum of squared residual from least squares

q2df

Current cost of portfolio: -0.3467


,Asset,Weight,Current Price,Cost Contribution
0,0,0.064375,31.230882,2.010476
1,1,0.272296,50.587255,13.774696
2,2,-0.079759,36.350000,-2.899249
3,3,-0.230416,71.693137,-16.519224
4,4,0.320271,39.039706,12.503300
5,5,0.012698,13.935294,0.176948
6,6,-0.303609,55.095400,-16.727482
7,7,0.118561,61.856863,7.333823


Therefore, the arbitrage strategy is to enter into a portfolio where the weights are chosen by the value of the least squares residual. i.e. to go long assets where the residual is negative (P1 - B1 @ psi1 < 0) and short assets where the residual is positive (P1 - B1 @ psi1 > 0). These weights are akin to *-resid* printed above. Specifically:

*   Buy 0.064 units of asset 0
*   Buy 0.272 units of asset 1
*   Sell 0.080 units of asset 2
*   Sell 0.230 units of asset 3
*   Buy 0.320 units of asset 4
*   Buy 0.013 units of asset 5

*  Sell 0.304 units of asset 6
*  Buy 0.119 units of asset 7.

to receive $0.3467 up front at T = 0 (xi.T @ P1 <0) and have a guaranteed payoff of 0 in all future states (xi.T @ B1 = 0) at T = 1. This is an example of type 2 arbitrage! i.e. exploits mispricing but has no risk in all future states and a negative cost we receive at t = 0. It also neatly highlights the use of the sum of squared residuals value in least squares estimation, as if this is > than zero, you can construct a portfolio where you collect an amount that is equal to residual1 at T = 0 and has zero at T = 1.




In [89]:
# Scale the portfolio for a profit of $1 (optional)
scaling_factor = 1 / (-initial_cost) # if you want to receive $1 today
xi_scaled = xi * scaling_factor
print("Scaled portfolio xi:", xi_scaled)
print("Scaled initial cost:", xi_scaled.T @ P1)
print("Scaled payoff:", xi_scaled.T @ B1)
print("Is payoff approximately zero in all future states?", np.allclose(xi_scaled.T @ B1, 0, atol=1e-8))

# below are the new weights if you chose to receive $1 today (instead of $0.3467), while all state payoffs in the future are still = zero

Scaled portfolio xi: [[ 0.18567148]
 [ 0.78536492]
 [-0.23004446]
 [-0.66457292]
 [ 0.92373774]
 [ 0.0366236 ]
 [-0.8756808 ]
 [ 0.3419583 ]]
Scaled initial cost: [[-1.]]
Scaled payoff: [[-1.19735333e-11 -9.27258270e-12  6.18172180e-12  3.34665629e-12
  -1.02318154e-11]]
Is payoff approximately zero in all future states? True


In [90]:
# df for $1 today, 0 payoff in future

weights2 = [0.18567148, 0.78536492, -0.23004446, -0.66457292, 0.92373774, 0.0366236, -0.8756808, 0.3419583]
q2df2 = pd.DataFrame({
    'Asset': range(8),   # Asset 0 through 7
    'Weight': weights2

})

q2df2['Current Price'] = P1
q2df2['Cost Contribution'] = q2df2['Weight']*q2df2['Current Price']

print(f"Current cost of portfolio: {q2df2['Cost Contribution'].sum():.4f}")
q2df2

Current cost of portfolio: -1.0000


,Asset,Weight,Current Price,Cost Contribution
0,0,0.185671,31.230882,5.798684
1,1,0.785365,50.587255,39.729455
2,2,-0.230044,36.350000,-8.362116
3,3,-0.664573,71.693137,-47.645318
4,4,0.923738,39.039706,36.062450
5,5,0.036624,13.935294,0.510361
6,6,-0.875681,55.095400,-48.245984
7,7,0.341958,61.856863,21.152468


In [85]:
#q3

q3 = pd.read_csv('Q3.csv')
q3 = q3.drop('Unnamed: 0', axis=1)
q3 = q3.reset_index().rename(columns={'index': 'Asset'})
q3

,Asset,State 1,State 2,State 3,Time 0 price
0,0,22.60,21.18,10.70,17.960784
1,1,94.96,56.93,44.99,59.758824
2,2,10.00,0.00,20.00,20.503555


In [86]:
P2 = q3[['Time 0 price']].values
B2 = q3[['State 1', 'State 2', 'State 3']].values

In [87]:
#calculating psi2 by least squares
psi2, residuals2, rank2, s2 = np.linalg.lstsq(B2, P2, rcond=None) # find state prices by least squares
print("psi2 (OLS):", psi2)
print("Residual2:", np.linalg.norm(B2 @ psi2 - P2))

#matrix result (same)
B2_T = B2.T
psi_matrix2 = np.linalg.inv(B2_T @ B2) @ B2_T @ P2

print("State prices via matrix inversion:", psi_matrix2.flatten())
print("Any negative state prices?", np.any(psi2 < 0))

#ψ has a negative component, there is no non-negative state price vector representing the assets' prices. This means arbitrage exists.

psi2 (OLS): [[-0.19756509]
 [ 0.491     ]
 [ 1.1239603 ]]
Residual2: 3.517007107451656e-14
State prices via matrix inversion: [-0.19756509  0.491       1.1239603 ]
Any negative state prices? True


In [88]:
# Identify indices of negative state prices in psi2
neg_states = np.where(psi2 < 0)[0]

# Loop over each negative state price
for i in neg_states:
    # Print the negative state price (adding 1 to match human-readable state numbering)
    print(f"Negative state price at state {i+1}: {psi2[i].item():.6f}")

    # Create unit vector e_i corresponding to the i-th state
    # This represents a payoff of 1 in state i, 0 elsewhere
    e_i = np.zeros(B2.shape[1])
    e_i[i] = 1

    # Solve B^T x = e_i for portfolio x that replicates 1 unit payoff in state i
    # This gives us the portfolio that delivers exactly e_i as its state payoff
    x = np.linalg.lstsq(B2.T, e_i, rcond=None)[0]

    # Calculate the initial cost of the portfolio: x^T * P. We want it to be < 0
    cost = x @ P2

    # Calculate the actual payoff across states: B^T * x. Should be 1 in the first state with a negative state price, 0 elsewhere
    payoff = B2.T @ x

    # Print the portfolio weights, cost, and resulting payoffs
    print(f"Portfolio weights x replicating state {i+1} payoff:\n{x}")
    print(f"Initial cost x^T P = ${cost.item():.6f}")
    print(f"State payoffs B^T x = {payoff}\n")

    print(f"Is payoff in State 1 equal to 1? = {np.allclose(payoff[0],1 )}")
    print(f"Are payoffs in State 2 and 3 ~ equal to zero? = {np.allclose(payoff[1:], 0)}")

Negative state price at state 1: -0.197565
Portfolio weights x replicating state 1 payoff:
[-0.1029911   0.03831638 -0.03109246]
Initial cost x^T P = $-0.197565
State payoffs B^T x = [ 1.00000000e+00 -7.12912260e-16 -6.66133815e-16]

Is payoff in State 1 equal to 1? = True
Are payoffs in State 2 and 3 ~ equal to zero? = True


Arbitrage strategy: We build a portfolio with weights of:

*   -0.103 of Asset 0 (short)
*   0.038 of Asset 1 (long)
*   -0.031 of Asset 2 (short)

That has negative cost at T = 0 ($0.1976 we receive) and pays off 1 in State 1, and 0 elsewhere (in State 2 and 3) at T = 1.

These are the weights that let us replicate a payoff of 1 in the first state and zero elsewhere.

Since this portfolio costs < 0 at T = 0 and pays off 1 in State 1 (and zero elsewhere) at T =1, the arbitrage strategy is to go long the replicating portfolio in the weights listed above. This allows us to generate a riskless profit given negative cost today (receive this amount) and non-negative payoff in the future at all possible states.
